In [9]:
from data_processing import *
import tensorflow as tf
import copy
import numpy as np

In [10]:
dialogues, ides = [], []
for i in range(0, 9):
    dialogues.append(preprocessed_dialogue(i))
    ides.append(preprocessed_ide(i))

In [11]:
dialogue_input, dialogue_output = sort(dialogues)
ide_input, ide_output = sort(ides)
maxLength = 0
# 21 is the max length
for i in dialogue_input:
    maxLength = max(maxLength, len(i))

In [12]:
one_hot_encoded_dialogue_input, one_hot_encoded_dialogue_output = np.array([]), np.array([])
sample = []
# Trying padding and processing
for i in dialogue_input:
    temp = i
    while (len(temp) < 21): temp.append(0)
    sample.append(temp)
    
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
sample = scaler.fit_transform(sample)

one_hot_encoded_dialogue_input = tf.keras.utils.to_categorical(sample)

for i in dialogue_output:
    temp = tf.keras.utils.to_categorical(i).tolist()
    while (len(temp) < 47):
        temp.append(0)
    one_hot_encoded_dialogue_output = np.append(one_hot_encoded_dialogue_output, temp)
# print(one_hot_encoded_dialogue_input[0])

In [18]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=21, activation='softmax'),
        tf.keras.layers.Dense(units=128, activation='softmax'),
        tf.keras.layers.Dense(units=1, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def train_and_evaluate_model(model, X, y, k=7):
    from sklearn.model_selection import KFold
    kfold = KFold(n_splits=k, shuffle=True, random_state=0)
    scores = []
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        score = model.evaluate(X_test, y_test, verbose=0)
        scores.append(score)
    print(scores)
    return np.mean(scores)

# print()
# Load the input data and one-hot encode it
X = one_hot_encoded_dialogue_input
y = one_hot_encoded_dialogue_output

model = build_model()
score = train_and_evaluate_model(model, X, y)
print("Cross-validated accuracy:", score)


[0.9765100479125977, 0.9831649661064148, 0.9595959782600403, 0.9831649661064148, 0.9764309525489807, 0.9865319728851318, 0.9831649661064148]
Cross-validated accuracy: 0.9783662642751422


In [16]:
def last_speaker

SyntaxError: invalid syntax (2888181176.py, line 1)